# Storing API Requests in MongoDB
What we want to do is to store the requests in such a way that we can make use of the objects and classes to analyse the league and results.  

This builds on the other notebooks that have written that show the basics of API calls and MongoDB. 

In [1]:
# Import packages 
import json
import os

import pandas as pd
import pymongo

import requests

In [2]:
# Get the database details from the conda environment 
mongodb_password = os.environ.get("mongodb_password")
mongodb_user = os.environ.get("mongodb_user")

When we end up automating this, we are going to want to set a hard limit and control the requests, throttling our access to make sure we don't incure any additional costs for the tier. 

In [3]:
# Define max requests for this run 
api_max_requests = 10

In [4]:
# Access the database client and define database 
client = pymongo.MongoClient("mongodb+srv://" + mongodb_user + ":" + mongodb_password + "@basiccluster-6s0er.mongodb.net/test?retryWrites=true&w=majority")
db = client.football

In [5]:
# Get the API details from the conda environment 
headers = {
    'x-rapidapi-host': os.environ.get("api_host"), 
    'x-rapidapi-key': os.environ.get("api_key") 
    }

def api_call(url):
    response = requests.request("GET", url, headers=headers)
    
    # Return a dictionary of the response text
    return json.loads(response.text)

For now, I'm going to focus on the premier league. If required, we can extend and restructure.

In [23]:
# Define the base URL for the API requests
base_url = 'https://api-football-v1.p.rapidapi.com/v2/'
premier_league_id = '524'
fixtures_url = base_url + 'fixtures/league/' + premier_league_id

# Call the league fixture list and compare to current version
fixtures = api_call(fixtures_url)

In [24]:
# Grab the particular fixture information, and remove the unnecessary response data
fixtures = fixtures['api']['fixtures']

In [21]:
db.fixtures.insert_many(fixtures)

/Users/willcanniford/anaconda3/envs/environment/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


[ObjectId('5dc48549808a6c58861183b1'),
 ObjectId('5dc48549808a6c58861183b2'),
 ObjectId('5dc48549808a6c58861183b3'),
 ObjectId('5dc48549808a6c58861183b4'),
 ObjectId('5dc48549808a6c58861183b5'),
 ObjectId('5dc48549808a6c58861183b6'),
 ObjectId('5dc48549808a6c58861183b7'),
 ObjectId('5dc48549808a6c58861183b8'),
 ObjectId('5dc48549808a6c58861183b9'),
 ObjectId('5dc48549808a6c58861183ba'),
 ObjectId('5dc48549808a6c58861183bb'),
 ObjectId('5dc48549808a6c58861183bc'),
 ObjectId('5dc48549808a6c58861183bd'),
 ObjectId('5dc48549808a6c58861183be'),
 ObjectId('5dc48549808a6c58861183bf'),
 ObjectId('5dc48549808a6c58861183c0'),
 ObjectId('5dc48549808a6c58861183c1'),
 ObjectId('5dc48549808a6c58861183c2'),
 ObjectId('5dc48549808a6c58861183c3'),
 ObjectId('5dc48549808a6c58861183c4'),
 ObjectId('5dc48549808a6c58861183c5'),
 ObjectId('5dc48549808a6c58861183c6'),
 ObjectId('5dc48549808a6c58861183c7'),
 ObjectId('5dc48549808a6c58861183c8'),
 ObjectId('5dc48549808a6c58861183c9'),
 ObjectId('5dc48549808a6c

We have inserted the 380 fixtures for the premier league for this season. Going forward we are going to want to call this again and then just update those that have changed i.e. games that have been played. 